In [6]:
import random
import math
import copy

In [7]:
class Test(CRO):
    """
    This is a derived class to find the minimum value of sin(x) + cos(y).
    In real terms, you should implement this class according to your real problem.
    Note that every copy operation is deep-copy.
    """
    def __init__(self, structure):
        CRO.__init__(self, structure)

    def wall(self, m):
        new = copy.deepcopy(m)
        new.structure[0], new.structure[1] = new.structure[1], new.structure[0]
        return new

    def dec(self, m):
        new1 = copy.deepcopy(m)
        new2 = copy.deepcopy(m)
        new1.structure[0] += random.random()
        new2.structure[1] += random.random()
        return [new1, new2]

    def inter(self, m1, m2):
        new1 = copy.deepcopy(m1)
        new2 = copy.deepcopy(m2)
        new1.structure[0] = m2.structure[0]
        new1.structure[1] = m1.structure[1]
        new2.structure[0] = m1.structure[0]
        new2.structure[1] = m2.structure[1]
        return [new1, new2]

    def syn(self, m1, m2):
        new = copy.deepcopy(m1)
        if random.random() < 0.5:
            new.structure[0] = m2.structure[0]
        else:
            new.structure[1] = m2.structure[1]
        return new

    def fit_func(self, m):
            return math.sin(m.structure[0]) + math.cos(m.structure[1])

NameError: name 'CRO' is not defined

In [ ]:
class Molecule:
    def __init__(self, structure):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal


class CRO:
    """
    A base class represent the CRO algorithm.
    You should inherit from this class.
    """
    optimal = None  # A Molecule object represent the optimal solution
    cnt = 0
    limit = 1000
    KELossRate = 0.2
    mole_coll = 0.2
    alpha = 100
    beta = 100
    buffer = 0
    init_ke = 100
    pop = []

    def __init__(self, structure):
        """
        * fit_func: Object function
        * structure: Initial solution list [s1, s2, ..., sn]
        """
        for s in structure:
            self.pop.append(Molecule(s))

        for mol in self.pop:
            # You should implement this function in your derived class
            mol.pe = self.fit_func(mol)
            mol.ke = self.init_ke
            mol.update()
            if self.optimal is None:
                self.optimal = copy.deepcopy(mol)
            elif mol.pe < self.optimal.pe:
                self.optimal = copy.deepcopy(mol)

    def run(self):
        while self.cnt < self.limit:
            self.cnt += 1
            if random.random() > self.mole_coll or len(self.pop) < 2:
                m = random.choice(self.pop)
                if m.num_of_hits > self.alpha:
                    self.decomposition(m)
                else:
                    self.on_wall(m)
            else:
                m1, m2 = random.sample(self.pop, 2)
                if m1.ke <= self.beta and m2.ke <= self.beta:
                    self.synthesis(m1, m2)
                else:
                    self.interaction(m1, m2)

    def update(self, m):
        """
        If m is the current optimal solution, save it to the optimal.
        """
        if m.pe < self.optimal.pe:
            self.optimal = copy.deepcopy(m)

    def decomposition(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.dec(m)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m.pe + m.ke - new1.pe - new2.pe
        if tmp >= 0 or tmp + self.buffer >= 0:
            if tmp >= 0:
                q = random.random()
                new1.ke = tmp * q
                new2.ke = tmp * (1 - q)
            else:
                new1.ke = (tmp + self.buffer) * random.random() * random.random()
                new2.ke = (tmp + self.buffer - new1.ke) * random.random() * random.random()
                self.buffer = self.buffer + tmp - new1.ke - new2.ke
            new1.update()
            new2.update()
            self.pop.remove(m)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def on_wall(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.wall(m)
        new.pe = self.fit_func(new)
        tmp = m.pe + m.ke - new.pe
        if tmp >= 0:
            q = random.uniform(self.KELossRate, 1)
            new.ke = tmp * q
            new.update()
            self.buffer += tmp * (1 - q)
            self.pop.remove(m)
            self.pop.append(new)
            self.update(new)

    def interaction(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.inter(m1, m2)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new1.pe - new2.pe
        if tmp >= 0:
            q = random.random()
            new1.ke = tmp * q
            new2.ke = tmp * (1 - q)
            new1.update()
            new2.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def synthesis(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.syn(m1, m2)
        new.pe = self.fit_func(new)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new.pe
        if tmp >= 0:
            new.ke = tmp
            new.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new)
            self.update(new)

In [ ]:
s = [[random.random(), random.random()] for i in range(10)]
print(s)
cro = Test(s)
cro.run()
print(cro.optimal.pe)

[[0.3800599818155599, 0.3856326529570886], [0.38334828547671174, 0.6661160456099557], [0.16275922329625947, 0.5398813400951701], [0.40516704337762066, 0.2241413123441386], [0.6220000707077118, 0.662084613055528], [0.7875041055456901, 0.6158477427109108], [0.7132308253786828, 0.9433686143530275], [0.05526775630777481, 0.18857035401206534], [0.762722551669103, 0.6028469108711643], [0.5415937330219721, 0.9746974678765357]]
-1.9990084893550102


# MY IMCRO

In [ ]:
# Initialization


In [ ]:
class MoleCule:
    def __init__(self, structure,supersequence):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)
        self.supersequence = copy.deepcopy(supersequence)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal

In [ ]:
def uni_moleReact():
    return
def inter_moleReact():
    return

In [ ]:
# Iteration
i = 0
moleColl = 0.2
alpha = random.randint(10, 100)
beta = random.randint(10, 100)
while i!=1000:
    i+=1
    t = random.random()
    if t > moleColl:
        uni_moleReact()
    else :
        inter_moleReact()
    

In [158]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original 
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = ''.join(copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1)))

    for i in range(len(copied_set_of_strings)):
        # print("i = ",i)
        counter = 0
        for j in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            # print("j and counter and supersequence length and inserted index",j," ",counter," ",len(supersequence)," ",inserted_pos)
            if inserted_pos == len(supersequence) or j != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,j,inserted_pos)
            counter = inserted_pos +1
            # print(supersequence)

    return supersequence

# supersequence_generate(['acg', 'cat', 'gtt','tgc'])

def population_generation(pop_size,set_of_strings):
    l=[]
    for i in range(pop_size):
        l.append(supersequence_generate(set_of_strings))
    return l


# Generate many supersequence with popsize = 10 and given set of strings
initial_population =  population_generation(10,['acg', 'cat', 'gtt','tgc'])
initial_population


['ctggtcactgat',
 'acatggtcctgt',
 'cagttacgct',
 'tacgcgtctat',
 'gttagctcgcat',
 'gtcatagccgt',
 'agtccgttcgat',
 'ctagaccgtgtt',
 'atccagtgct',
 'cagtactttggc']

In [11]:
def encoding_population(initial_population):
    dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
    l=[]
    for i in initial_population:
        k=''
        for j in i:
            k+=str(dict[j])
        l.append(k)
    return l
encoding_population(initial_population)

['1021213023',
 '3021120312',
 '1010232331',
 '10032313312',
 '0310122321',
 '103021132233',
 '201212013',
 '21301323321',
 '031232133',
 '21033013231']